In [45]:
import subprocess
import random
import os
from pathlib import Path

# bug patterns are defined here
# NOTE:
#  1. `src` and `dst` will be concatenated to form a substitution command for `sed`
#  2. the first character of `src` will be considered as the seperator for the substitution command
#  3. pay attention to the escaped sequence in python string
#  4. Example: ['/\(i *\)++', '\\1--'] will form the following command
#                     s/\(i *\)++/\1--/

buglist = [  # TODO: add more bug patterns
  #   src         dst
  ['/reg', '\\wire'],
]

#===========================================
# common functions

def shell(cmd):   # execute the shell command and return the output as a string
    output = subprocess.Popen("bash -c \"" + cmd + "\"", stdout = subprocess.PIPE, shell = True).communicate()
    return output[0].decode('UTF-8')

def shellv(cmd):  # return array of lines
    res = shell(cmd).split('\n')
    if res[-1] == '':
        res = res[:-1]
    return res


#===========================================
# check project

# if os.getenv('RTL_HOME') == None:
#     print("Error: RTL_HOME is not set!")
#     exit(-1)

# RTL_HOME = Path(os.getenv('RTL_HOME'))
RTL_HOME = "/home/shiroha/LLM/Verilog-Auto-Debug/RTLLM"
ERRRO_HOME  = "/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate"

print("RTL_HOME=" + str(RTL_HOME))

def check_path(path):
    if not path.exists():
        print("Error: " + str(path) + " does not exist!")
        exit(-1)

# ACCU_HOME = RTL_HOME / 'accu'
# ALU_HOME = RTL_HOME / 'alu'
# FSM_HOME = RTL_HOME / 'fsm'
# PE_HOME = RTL_HOME / 'pe'
# RAM_HOME = RTL_HOME / 'RAM'

# for p in [ACCU_HOME, ALU_HOME, FSM_HOME, PE_HOME, RAM_HOME]:
#     check_path(p)


RTL_HOME=/home/shiroha/LLM/Verilog-Auto-Debug/RTLLM


In [48]:
import re 
#===========================================
# compute filelist

tb_filelist   = shellv("find " + str(RTL_HOME + "/") + " -name '*.v' | grep tb.v")

v_filelist    = shellv("find " + str(RTL_HOME + "/") + " -name '*.v' | grep -vE 'tb.v|testbench.v' ")

# print(v_filelist)

for f in v_filelist:
    check_path(Path(f))
    filename = shell("basename " + f +  " | cut -d'.' -f1")
    filename = re.sub(r'[\n\r]+', '', filename)   # 使用 re.sub 替换文件名中的换行符
    cp_dst_dir = os.path.join(str(ERRRO_HOME), filename)
    shell("mkdir -p " + cp_dst_dir)
    cp_dst_file = os.path.join(cp_dst_dir, filename + '.v')
    shell("cp '" + f + "' '" + cp_dst_file + "'")
    # print(f)


e_filelist = shellv("find " + str(ERRRO_HOME) + "/"+ " -name '*.v' ")

e_filelist

['/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_multi_16bit/verified_multi_16bit.v',
 '/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_JC_counter/verified_JC_counter.v',
 '/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_adder_16bit/verified_adder_16bit.v',
 '/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_multi_pipe_8bit/verified_multi_pipe_8bit.v',
 '/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_calendar/verified_calendar.v',
 '/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_multi_pipe_4bit/verified_multi_pipe_4bit.v',
 '/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_div_16bit/verified_div_16bit.v',
 '/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_signal_generator/verified_signal_generator.v',
 '/home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_freq_d

In [ ]:
# 随机插入版
# #===========================================
# # random select a file and a bug

# def rand_element(array):
#     i = random.randint(0, len(array) - 1)
#     return array[i]

# filename = None
# bug = None

# while True:
#     filename = rand_element(e_filelist)
#     bug = rand_element(buglist)
#     # check if we can find the bug pattern in the file
#     regex = bug[0][1:]
#     grep_cmd = "grep -n '" + regex + "' " + filename
#     grep_results = shellv(grep_cmd)

#     if grep_results != []:
#         break
#     # if we can not find the bug pattern in the file, select again

# #===========================================
# # select a random matched line and prepare the `sed` command to insert the bug

# grep_random = rand_element(grep_results)
# line = grep_random.split(':')[0]

# def sed_str(bug):
#     seperator = bug[0][0]
#     return bug[0] + bug[0][0] + bug[1] + bug[0][0]

# sed_cmd = "sed -e '" + line + "s" + sed_str(bug) + "' " + filename

# #===========================================
# # display the `sed` command and different after insertion for debug

# # if os.getenv('DEBUG') != None:
# print(sed_cmd)
# print(shell(sed_cmd + ' | diff ' + filename + ' -'))

# #===========================================
# # apply the `sed` command to insert the bug
# sed_cmd = "sed -i" + sed_cmd[3:]
# shellv(sed_cmd)

# print("Insert 1 bug somewhere sucessfully!")

In [54]:
#===========================================
# random select a file and a bug

def rand_element(array):
    i = random.randint(0, len(array) - 1)
    return array[i]

def sed_str(bug):
    seperator = bug[0][0]
    return bug[0] + bug[0][0] + bug[1] + bug[0][0]

filename = None
bug = None

for f in e_filelist:
    check_path(Path(f))
    filename = shell("basename " + f +  "| cut -d'.' -f1")
    filename = re.sub(r'[\n\r]+', '', filename)   # 使用 re.sub 替换文件名中的换行符
    print("Insert bug in " + str(filename))

    for bugtype in range(len(buglist)):
        bug   = buglist[bugtype]
        regex = bug[bugtype][1:]
        grep_cmd = "grep -n '" + regex + "' " + f
        grep_results = shellv(grep_cmd)  # 寻找所有有指定元素的行
    
        if grep_results == []:
            continue                # 没找着就跳过，寻找下一个类型的bugtype

        for grep_result in grep_results: 
            line = grep_result.split(':')[0] 
            generate_file = os.path.join(os.path.dirname(f), filename + '_bugtype' + str(bugtype) + '_line' + line + '.v')
            # print(generate_file)
            shell("cp " + f + " " + generate_file) 
            sed_cmd = "sed -e '" + line + "s" + sed_str(bug) + "' " + generate_file  # 创建文件并写入bug
            # -i 选项指示 sed 直接修改文件，而不是输出修改到标准输出 -e 输出到stdout
            
            print(sed_cmd)
            # if os.getenv('DEBUG') != None:
            print(shell(sed_cmd + ' | diff ' + os.path.join(os.path.dirname(f), filename) + ' -'))
            # shellv(sed_cmd)


Insert bug in verified_multi_16bit
sed -e '12s/reg/\wire/' /home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_multi_16bit/verified_multi_16bit_bugtype0_line12.v
12c12
< reg [15:0] areg;    // Multiplicand a register.
---
> wire [15:0] areg;    // Multiplicand a register.
50c50
<             yout_r <= yout_r + ({16'h0000, bwire} << (i-1)); // Accumulate and shift
---
>             yout_r <= yout_r + ({16'h0000, breg} << (i-1)); // Accumulate and shift

sed -e '13s/reg/\wire/' /home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_generate/verified_multi_16bit/verified_multi_16bit_bugtype0_line13.v
13c13
< reg [15:0] breg;    // Multiplier b register.
---
> wire [15:0] breg;    // Multiplier b register.
50c50
<             yout_r <= yout_r + ({16'h0000, bwire} << (i-1)); // Accumulate and shift
---
>             yout_r <= yout_r + ({16'h0000, breg} << (i-1)); // Accumulate and shift

sed -e '14s/reg/\wire/' /home/shiroha/LLM/Verilog-Auto-Debug/ErrorSet/Errordata_g